# MASTER - Notebook 4
## Hierarchical clustering with second procedure
### Matteo Grazioso 884055

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from datetime import datetime
import json
import os
import folium
import warnings
warnings.filterwarnings('ignore')

import myfunctions as mf # Custom functions

In [ ]:
# Disply all columns and all rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

---------


In [ ]:
# Open the dataset that has also the geo coordinates
# Find all txt files in the data folder
txt_file = mf.find_txt_files('data/processed/data-GTFS')

# From the list of txt files, remove the files that don't contain geo coordinates that are all json files and stops.txt
for file in txt_file:
    if file.endswith('stops.txt'):
        txt_file.remove(file)
    elif file.endswith('.json'):
        txt_file.remove(file)

print("Select a dataset with geo coordinates from the list:")

# Choose a dataset from the list of txt files
selected_dataset = mf.choose_dataset(txt_file)

if selected_dataset:
    print(f"You selected the dataset {selected_dataset}")
else:
    print("No dataset selected.")

path  = selected_dataset

In [ ]:
df = pd.read_csv(path, header=0, sep='\t')

# Save the name of the file in a variable for future use extracting the name of the file from the path
file_name = path.split('/')[3]
file_name = file_name.split('.')[0]

subfolder = file_name
print(f"File name: {file_name}")

# Display the first 5 rows of the dataframe
df.head()

# Convert the column 'DATA' to datetime format
df['DATA'] = pd.to_datetime(df['DATA'], format='%Y-%m-%d')

In [ ]:
df.head()

In [ ]:
# Print the interval of dates for which we have data
print('Date range: {} to {}'.format(df['DATA'].min(), df['DATA'].max()))

In [ ]:
# Rename stop_lat to LONGITUDE and stop_lon to LATITUDE
df.rename(columns={'stop_lat': 'LATITUDE', 'stop_lon': 'LONGITUDE'}, inplace=True)

In [ ]:
# Print the number of unique values of the column 'FERMATA'
print('Number of unique values of the column FERMATA: {}'.format(df['FERMATA'].nunique()))

In [ ]:
# For each stop, store the number of use for each ticket code
# Each stop is a point identified by the coordinates (latitude, longitude)

with open('data/dictionaries/dict_ticket_codes.json') as f:
        ticket_codes = json.load(f)

print('The ticket codes are: ', ticket_codes)

# Change 5-STUD, 6-STUD to STUD in the dataframe
df['TICKET_CODE'] = df['TICKET_CODE'].replace(['5-STUD', '6-STUD'], 'STUD')
# Change 5-WKRS, 6-WKRS to WKRS in the dataframe
df['TICKET_CODE'] = df['TICKET_CODE'].replace(['5-WKRS', '6-WKRS'], 'WKRS')
# Change 5-RET, 6-RET to RET in the dataframe
df['TICKET_CODE'] = df['TICKET_CODE'].replace(['5-RET', '6-RET'], 'RET')

# Print the unique ticket codes
# Print information about the changes made
print('The ticket codes 5-STUD and 6-STUD have been changed to STUD')
print('The ticket codes 5-WKRS and 6-WKRS have been changed to WKRS')
print('The ticket codes 5-RET and 6-RET have been changed to RET')

# Convert all the ticket codes to string
df['TICKET_CODE'] = df['TICKET_CODE'].astype(str)

ticket_codes = df['TICKET_CODE'].unique()
# Sort the ticket codes
ticket_codes.sort()

print('The considered ticket codes are: ', ticket_codes)

# For each stop, store the number of visits for each ticket code
# Iterate over the stops dataframe and for each stop, store the number of visits for each ticket code
# Notice that a stop is a pair of coordinates (latitude, longitude)

# The columns of the dataframe are:
# ['DATA', 'ORA', 'DATA_VALIDAZIONE', 'SERIALE', 'FERMATA', 'DESCRIZIONE',
#        'TITOLO', 'TICKET_CODE', 'DESCRIZIONE_TITOLO', 'LATITUDE', 'LONGITUDE']

# Create a dataframe of stops
df_stop = df[['LATITUDE', 'LONGITUDE', 'TICKET_CODE', 'FERMATA', 'DESCRIZIONE']]

df_stop.head()


In [ ]:
df.head()

In [ ]:
df_stop_count = df_stop.groupby(['LATITUDE', 'LONGITUDE', 'TICKET_CODE', 'FERMATA', 'DESCRIZIONE']).size().reset_index(name='COUNT')

df_stop_count.head(20)

In [ ]:
# Describe the column COUNT
df_stop_count['COUNT'].describe()

In [ ]:
# Pivot table for the ticket codes but maintaining the information about the stops
df_stop_count = df_stop_count.pivot_table(index=['LATITUDE', 'LONGITUDE', 'FERMATA', 'DESCRIZIONE'], columns='TICKET_CODE', values='COUNT', fill_value=0)
df_stop_count.reset_index(inplace=True)

# For each stop (LATITUDE, LONGITUDE), change the counter of each ticket code as a percentage of the total number of tickets
for row in range(len(df_stop_count)):
    total = df_stop_count.iloc[row, 4:].sum()
    for col in range(4, len(df_stop_count.columns)):
        df_stop_count.iloc[row, col] = df_stop_count.iloc[row, col] / total * 100

In [ ]:
df_stop_count

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def haversine_distance(coord1, coord2):
    """
        Calculate the distance between two points on Earth using the haversine formula.
        The haversine formula determines the great-circle distance between two points on a sphere given their longitudes and latitudes.
        The haversin formula is specified as:
            a = sin²(Δlat/2) + cos(lat1).cos(lat2).sin²(Δlong/2)
            c = 2.atan2(√a, √(1−a))
            d = R.c
        where:
            lat1, long1 = Latitude and Longitude of point 1 (in decimal degrees)
            lat2, long2 = Latitude and Longitude of point 2 (in decimal degrees)
            R = Radius of the Earth in kilometers
            Δlat = lat2− lat1
            Δlong = long2− long1

        :param coord1: Tuple of (latitude, longitude) for point 1
        :param coord2: Tuple of (latitude, longitude) for point 2
        :return: Distance between the two coordinates in kilometers
    """
    lon1, lat1 = coord1
    lon2, lat2 = coord2
    
    R = 6371  # Radius of the Earth in kilometers
    
    # Convert decimal degrees to radians
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    
    # Apply haversine formula
    # a = sin²(Δlat/2) + cos(lat1).cos(lat2).sin²(Δlong/2)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2

    # c = 2.atan2(√a, √(1−a))
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # d = R.c
    distance = R * c

    # print('Distance between stops: ', distance)
    return distance

def cosine_similarity_distance(counts1, counts2):
    # Calculate cosine similarity between two count vectors
    """
        Calculate the cosine similarity between two count vectors.
        The cosine similarity is a measure of similarity between two non-zero vectors of an inner product space 
        that measures the cosine of the angle between them.
        The cosine of 0° is 1, and it is less than 1 for any other angle.
        It is thus a judgement of orientation and not magnitude: two vectors with the same orientation have a cosine similarity of 1,
        two vectors at 90° have a similarity of 0, and two vectors diametrically opposed have a similarity of -1, independent of their magnitude.
        The cosine similarity is particularly used in positive space, where the outcome is neatly bounded in [0,1].
        The cosine similarity is mathematically defined as:
            cos(θ) = A.B / ||A||.||B||
        where:
            A.B = Dot product of A and B
            ||A|| = Euclidean norm of A
            ||B|| = Euclidean norm of B
        :param counts1: List of counts for each ticket code for stop 1
        :param counts2: List of counts for each ticket code for stop 2
        :return: Cosine similarity between the two count vectors

        # This function calls the cosine_similarity function from sklearn.metrics.pairwise to calculate the cosine similarity between two count vectors.
    """
    similarity_matrix = cosine_similarity([counts1], [counts2])
    similarity = 1 - similarity_matrix[0][0]
    
    # print('Similarity between stops: ', similarity)
    return similarity

def custom_distance(stop1, stop2, coord_weight, similarity_weight):
    """
        Calculate the custom distance between two stops.
        The custom distance is a weighted combination of the haversine distance between the two stops and the cosine similarity between the two stops.
        The custom distance is mathematically defined as:
            custom_distance = coord_weight * haversine_distance + similarity_weight * cosine_similarity_distance
        where:
            coord_weight = Weight for haversine distance
            similarity_weight = Weight for cosine similarity distance
        :param stop1: Tuple of (latitude, longitude, counts for each ticket code) for stop 1
        :param stop2: Tuple of (latitude, longitude, counts for each ticket code) for stop 2
        :param coord_weight: Weight for haversine distance
        :param similarity_weight: Weight for cosine similarity distance
        :return: Custom distance between the two stops
    """
    # Calculate distances
    # Calculate haversine distance between two stops
    coord_distance = haversine_distance((stop1[0], stop1[1]), (stop2[0], stop2[1]))
    # Calculate cosine similarity between two stops
    count_similarity = cosine_similarity_distance(stop1[4:], stop2[4:]) # CAMBIATO PERCHÉ NON C'È LA COLONNA 'FERMATA' e 'DESCRIZIONE'
    
    # Combine distances with appropriate weights
    combined_distance = coord_weight * coord_distance + similarity_weight * count_similarity
    return combined_distance

In [ ]:
# From df_stop_count remove descrizione
df_stop_count = df_stop_count.drop(columns=['DESCRIZIONE'])

In [ ]:
# df_count_stop is a df with columns [' LATITUDE ', ' LONGITUDE ', '1', '2', '3', '4', '5', '6', '7', 'STUD', 'RET', 'WKRS'] 
data = df_stop_count.values 
# print(data)

# Custom distance function parameters
coord_weight = 0.3
similarity_weight = 0.7

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

# Calculate linkage matrix using custom distance function
# The custom distance function is a weighted average of the haversine distance between coordinates and the similarity between the stop counts
linkage_matrix = linkage(data, method='single', metric=lambda x, y: custom_distance(x, y, coord_weight, similarity_weight))

# Create a dendrogram for visualization purposes 
plt.figure(figsize=(10, 5))
dendrogram(linkage_matrix, truncate_mode='lastp', p=12, leaf_rotation=90., leaf_font_size=10., show_contracted=True)

# Determine clusters based on a desired threshold or number of clusters
threshold = 0.25
clusters = fcluster(linkage_matrix, t=threshold, criterion='distance', depth=2, R=None, monocrit=None)

In [ ]:
# Adding cluster labels to the stop count dataframe
df_stop_count['Cluster'] = clusters
print(df_stop_count.shape)
df_stop_count.head()

In [ ]:
# get hex codes of tab_20 palette in the format 1:hex_code dict
cluster_colors = dict(zip(range(1,51), sns.color_palette("tab20", 50).as_hex()))

In [ ]:
# Countplot of cluster distribution 
fig = plt.figure(figsize=(10, 5))
sns.countplot(x='Cluster', data=df_stop_count, palette=cluster_colors)
plt.yticks(np.arange(0, max(df_stop_count['Cluster'].value_counts()) + 5, 2))
plt.title('Cluster Distribution: ' + 'date range: ' + str(df['DATA'].min().date()) + ' to ' + str(df['DATA'].max().date()))
plt.show()

In [ ]:
# Create a folium map centered around the mean latitude and longitude
center_lat = np.mean(df_stop_count['LATITUDE'])
center_lon = np.mean(df_stop_count['LONGITUDE'])
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Create markers for each stop and color them based on clusters
for idx, row in df_stop_count.iterrows():
    # Retrieve Cluster id and set it as a integer
    cluster = int(row['Cluster'])

    cluster_color = cluster_colors.get(cluster, 'gray')  # Default to gray if cluster color is not defined

    # cluster_color = cluster_
    # colors.get(cluster, 'gray')  # Default to gray if cluster color is not defined
    # Add a label to the marker with the name of the stop contained in the column 'DESCRIZIONE'
    # Retrieve the name of the stop from the column 'DESCRIZIONE' of the dataframe df matching the FERMATA
    location = df[df['FERMATA'] == row['FERMATA']]['DESCRIZIONE'].values[0]
    # Retrieve FERMATA id and set it as a integer
    fermata = int(row['FERMATA'])
    # Retrieve Cluster id and set it as a integer
    cluster = int(row['Cluster'])

    # Create a string for popup message containing the name of the stop, FERMATA and the cluster in BOLD
    popup_message = f"<b><i>{location}</i></b><br><br><u>Fermata:</u> {fermata}<br><u>Cluster:</u> {cluster}"

    # Add a marker to the map with the popup message and the color of the cluster
    # folium.Marker(location=[row['LATITUDE'], row['LONGITUDE']], popup=popup_message, icon=folium.Icon(color=cluster_color)).add_to(m)
    if fermata < 0:
        folium.Marker(location=[row['LATITUDE'], row['LONGITUDE']], popup=popup_message, icon=folium.Icon(color='white', icon_color=cluster_color, icon='bus', prefix='fa')).add_to(m)
    else:
        folium.Marker(location=[row['LATITUDE'], row['LONGITUDE']], popup=popup_message, icon=folium.Icon(color='white', icon_color=cluster_color, icon='ship', prefix='fa')).add_to(m)

m

In [ ]:
# print the number of unique values of the column 'FERMATA'
print('Number of unique values of the column FERMATA: {}'.format(df['FERMATA'].nunique()))
# print the numer of clusters
print('Number of clusters: {}'.format(df_stop_count['Cluster'].nunique()))
# Print the average number of stops per cluster
print('Average number of stops per cluster: {}'.format(df_stop_count['Cluster'].value_counts().mean()))


In [ ]:
# For each Cluster id, print the number of stops in that cluster, the FERMATA id and the name of the stop
df_stop_count_copy = df_stop_count.copy()
# To df_stop_count add a column with the name of the stop matching the FERMATA
df_stop_count_copy['DESCRIZIONE'] = df_stop_count_copy['FERMATA'].apply(lambda x: df[df['FERMATA'] == x]['DESCRIZIONE'].values[0])

# Consider only the columns ['Cluster', 'FERMATA', 'DESCRIZIONE']
df_stop_count_copy = df_stop_count_copy[['Cluster', 'FERMATA', 'DESCRIZIONE']]
df_stop_count_copy.columns.name = None
df_stop_count_copy.head()

print('Number of clusters: ', max(df_stop_count_copy['Cluster']))
print('\n')
for cluster in range(1, max(df_stop_count_copy['Cluster']) + 1):
    print(f"Cluster {cluster} contains {len(df_stop_count_copy[df_stop_count_copy['Cluster'] == cluster])} stops.")
    print(df_stop_count_copy[df_stop_count_copy['Cluster'] == cluster][['FERMATA', 'DESCRIZIONE']])
    print('\n')



In [ ]:
# Export the map to an HTML file
# Import the necessary libraries
from datetime import datetime
time = datetime.now().strftime("%Y%m%d-%H%M%S")
try:
    m.save('map/' + file_name + '_clustered_' + time + '.html')
    print('Map saved in map/' + file_name + '/' + file_name + '_clustered_' + time + '.html')
except:
    # Create a new folder in map folder
    os.mkdir('map/' + file_name)
    m.save('map/' + file_name + '/' + file_name + '_clustered_' + time + '.html')
    print('Map saved in map/' + file_name + '/' + file_name + '_clustered_' + time + '.html')

### Apply hierarchical clustering to the data of the main island of Venice (the most populated cluster)

In [ ]:
# Select only the data of the main island of Venice (the cluster most populated)
cluster_main_island = df_stop_count['Cluster'].value_counts().index[0]

print('The cluster with the highest number of stops is the cluster number', cluster_main_island)

cluster_main_island = df_stop_count[df_stop_count['Cluster'] == cluster_main_island]


In [ ]:
cluster_main_island

In [ ]:
# Apply hierarchical clustering to the data of the main island of Venice (the most populated one)

data = cluster_main_island.values

# Custom distance function parameters
coord_weight = 0.3
similarity_weight = 0.7

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

# Calculate linkage matrix using custom distance function
# The custom distance function is a weighted average of the haversine distance between coordinates and the similarity between the stop counts
linkage_matrix = linkage(data, method='single', metric=lambda x, y: custom_distance(x, y, coord_weight, similarity_weight))

# Create a dendrogram for visualization purposes 
dendrogram(linkage_matrix, truncate_mode='lastp', p=12, leaf_rotation=90., leaf_font_size=10., show_contracted=True)

# Determine clusters based on a desired threshold or number of clusters
threshold = 0.25
clusters = fcluster(linkage_matrix, t=threshold, criterion='distance', depth=2, R=None, monocrit=None)


In [ ]:
cluster_main_island

In [ ]:
cluster_main_island['Cluster'] = clusters


In [ ]:
# Create a folium map centered around the mean latitude and longitude
center_lat = np.mean(cluster_main_island['LATITUDE'])
center_lon = np.mean(cluster_main_island['LONGITUDE'])
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Create markers for each stop and color them based on clusters
for idx, row in cluster_main_island.iterrows():
    # Retrieve Cluster id and set it as a integer
    cluster = int(row['Cluster'])

    cluster_color = cluster_colors.get(cluster, 'gray')  # Default to gray if cluster color is not defined

    # cluster_color = cluster_
    # colors.get(cluster, 'gray')  # Default to gray if cluster color is not defined
    # Add a label to the marker with the name of the stop contained in the column 'DESCRIZIONE'
    # Retrieve the name of the stop from the column 'DESCRIZIONE' of the dataframe df matching the FERMATA
    location = df[df['FERMATA'] == row['FERMATA']]['DESCRIZIONE'].values[0]
    # Retrieve FERMATA id and set it as a integer
    fermata = int(row['FERMATA'])
    # Retrieve Cluster id and set it as a integer
    cluster = int(row['Cluster'])

    # Create a string for popup message containing the name of the stop, FERMATA and the cluster in BOLD
    popup_message = f"<b><i>{location}</i></b><br><br><u>Fermata:</u> {fermata}<br><u>Cluster:</u> {cluster}"

    # Add a marker to the map with the popup message and the color of the cluster
    # folium.Marker(location=[row['LATITUDE'], row['LONGITUDE']], popup=popup_message, icon=folium.Icon(color=cluster_color)).add_to(m)
    if fermata < 0:
        folium.Marker(location=[row['LATITUDE'], row['LONGITUDE']], popup=popup_message, icon=folium.Icon(color='white', icon_color=cluster_color, icon='bus', prefix='fa')).add_to(m)
    else:
        folium.Marker(location=[row['LATITUDE'], row['LONGITUDE']], popup=popup_message, icon=folium.Icon(color='white', icon_color=cluster_color, icon='ship', prefix='fa')).add_to(m)



m